In [38]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error

In [39]:
# Load the historical data
data = pd.read_csv('historical_data.csv')

In [40]:
# Filter the required columns and sort by date and ticker
data = data[['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Ticker']]
data['Date'] = pd.to_datetime(data['Date'])
data.sort_values(['Ticker', 'Date'], inplace=True)

In [41]:
# Encode the Ticker column
encoder = OneHotEncoder(sparse=False)
tickers_encoded = encoder.fit_transform(data[['Ticker']])

c:\Users\astev\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [42]:
# Add the encoded tickers as additional features
encoded_columns = [f'Ticker_{i}' for i in range(tickers_encoded.shape[1])]
encoded_df = pd.DataFrame(tickers_encoded, columns=encoded_columns, index=data.index)
data = pd.concat([data.reset_index(drop=True), encoded_df], axis=1)

In [43]:
# Scale numerical features (Open, High, Low, Close, Volume)
scaler = MinMaxScaler(feature_range=(0, 1))
numerical_features = ['Open', 'High', 'Low', 'Close', 'Volume']
data[numerical_features] = scaler.fit_transform(data[numerical_features])

In [44]:
# Prepare data for LSTM
def preprocess_data(data, n_steps=10):
    X, y = [], []
    feature_columns = numerical_features + encoded_columns
    for ticker in data['Ticker'].unique():
        ticker_data = data[data['Ticker'] == ticker]
        ticker_data = ticker_data[feature_columns].values
        for i in range(n_steps, len(ticker_data)):
            X.append(ticker_data[i - n_steps:i, :])  # Last n_steps rows as features
            y.append(ticker_data[i, 3])  # Predict 'Close' price
    return np.array(X), np.array(y)

In [45]:
# Define the number of timesteps
n_steps = 10


In [46]:
# Preprocess the data
X, y = preprocess_data(data, n_steps=n_steps)

In [47]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [48]:
# Build the LSTM model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25, activation='relu'),
    Dense(1)  # Output layer for predicting 'Close' price
])

In [49]:
# Compile the model
model.compile(optimizer='adam', loss='mse')

In [50]:
# Train the model
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.1)

Epoch 1/30
1083/1083 [==============================] - 12s 8ms/step - loss: 4.4818e-04 - val_loss: 5.1801e-05
Epoch 2/30
1083/1083 [==============================] - 11s 10ms/step - loss: 1.5265e-04 - val_loss: 5.5546e-05
Epoch 3/30
1083/1083 [==============================] - 11s 10ms/step - loss: 1.2501e-04 - val_loss: 5.5960e-05
Epoch 4/30
1083/1083 [==============================] - 10s 9ms/step - loss: 1.0300e-04 - val_loss: 4.9240e-05
Epoch 5/30
1083/1083 [==============================] - 10s 9ms/step - loss: 1.0880e-04 - val_loss: 2.8617e-05
Epoch 6/30
1083/1083 [==============================] - 10s 9ms/step - loss: 9.5229e-05 - val_loss: 3.7330e-05
Epoch 7/30
1083/1083 [==============================] - 11s 10ms/step - loss: 7.7481e-05 - val_loss: 3.6136e-05
Epoch 8/30
1083/1083 [==============================] - 10s 9ms/step - loss: 8.0773e-05 - val_loss: 2.6386e-05
Epoch 9/30
1083/1083 [==============================] - 10s 10ms/step - loss: 8.1869e-05 - val_loss: 1.4357e-

In [51]:
# Evaluate the model on the test set
loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")


301/301 [==============================] - 1s 3ms/step - loss: 1.5767e-04
Test Loss: 0.00015766863361932337


In [52]:
# Make predictions
predictions = model.predict(X_test)

301/301 [==============================] - 1s 3ms/step


In [53]:
# Inverse scale the predictions and actual values
y_test_scaled = scaler.inverse_transform(
    np.concatenate((np.zeros((y_test.shape[0], 4)), y_test.reshape(-1, 1)), axis=1))[:, -1]
predictions_scaled = scaler.inverse_transform(
    np.concatenate((np.zeros((predictions.shape[0], 4)), predictions), axis=1))[:, -1]

In [54]:
# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test_scaled, predictions_scaled))
print(f"Root Mean Squared Error (RMSE): {rmse}")

Root Mean Squared Error (RMSE): 19384829.941692118


In [55]:
# Calculate MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [56]:
mape = mean_absolute_percentage_error(y_test_scaled, predictions_scaled)
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

Mean Absolute Percentage Error (MAPE): 65.40%
